# H1 Big Data - Term Assignment 1
__Prof. Dr. Fabian Transchel, Hochschule Harz, 29.04.2021__
ftranschel@hs-harz.de

Submission due May 30, 12pm CEST. (Hand in via e-Mail to ftranschel@hs-harz or upload to StudIP.)

## Assignment and Logistics

__Grading__

The present assigment will be 40% of the overall grade of the course. The second assignment will comprise the other 60% of the grade.

__Preface and data generation__

The assigment is comprised of a set of questions being posed on a dataset created on the fly in the preface of this Jupyter notebook. The code may not be altered and the random seed must stay the same. On top of that, the dataset is saved to disc so if you would want to use the R language instead of Python, the dataset may be imported into R.

The submission must follow these guidelines: If written in the Python language, it is highly suggested to keep the notebook format. If notebook is not kep, the submitted code must be executable nonetheless. Each line of code and/or expression is to be explained in sufficient detail.

## Dataset preparation
<div style="font-weight:bold; color:red;">DO NOT ALTER THIS SECTION</div>

The purpose of this section is to create a substantially big (albeit synthetic) dataset without necessity to store or download it on the internet. It shall reside on your local machine only. To this end, it is absolutely mandatory you do not change the random seed to ensure correctness of the created dataset.

We will synthesize a big customer database with some structured as well as unstructured parts. You will then be tasked to answer questions about this dataset.

In [50]:
import numpy as np
import pandas as pd
import time

prng = np.random.RandomState(987654321) # Do NOT change this
df_users = pd.DataFrame(columns = ["ID","Surname","Name","Age","Subscription Date"])
number = 100000
names = ["Hans","Jordi","Franz","Timothy","Agaba","Ali","Sarah","Josie","Robert","Francine","Anna","Zoe","Simon","Thomas","Andreas","Alok","Lee","Jean-Luc"]
surnames = ["Mueller","Meier","Smith","Gwahsi","Thronton","Wellington","Stephenson","Pomme","Di Lillo","Bond","Kirk","Picard","Roth","Beierlorzer"]

def normalize_age(age,lower=16.0,upper=99.0):
    if(age < lower):
        return lower
    elif(age > upper):
        return upper
    return age

start = time.time()
userid = list(range(1,number+1))
surname = prng.choice(surnames,number)
name = prng.choice(names,number)
age = prng.normal(35.,10.,number)
# Normalize age to between 16 and 99:
na = np.vectorize(normalize_age)(age) # ok, but slow
sub_date = 1588150183 + prng.normal(10000.,5000.,number)
df_users = pd.DataFrame(np.array([userid,surname,name,na,sub_date]).T.tolist())
df_users.columns = ['ID','Surname','Name','Age','Subscription_Date']
print(df_users.info())

df_users.to_csv("user_table.csv") # You may import this file to R if you wish to work with that language.

# Next, we will create a set of posts for each user. To this end, we will synthesize text from the "Lorem_ipsum.txt" file.

with open("Lorem_ipsum.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

post_number = 1000000
post_id = list(range(1,post_number+1))
snippet_index = list(range(len(content)))
posts = prng.choice(snippet_index,post_number,replace=True)
posts_uids = prng.randint(1,number,post_number)
posts_date = 1588150183 + 5000000 + prng.normal(100000.,50000.,post_number)
posts_content = [ content[i] for i in posts]
df_posts = pd.DataFrame(np.array([post_id,posts_uids,posts_content,posts_date]).T.tolist())
df_posts.columns = ['PostID','UserID','Content','Timestamp']
df_posts.info()
df_posts.to_csv("postings_table.csv")
end = time.time()
print("Data Generation took ",(end - start)," seconds.")

## This is the end of the data generation.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ID                 100000 non-null  object
 1   Surname            100000 non-null  object
 2   Name               100000 non-null  object
 3   Age                100000 non-null  object
 4   Subscription_Date  100000 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   PostID     1000000 non-null  object
 1   UserID     1000000 non-null  object
 2   Content    1000000 non-null  object
 3   Timestamp  1000000 non-null  object
dtypes: object(4)
memory usage: 30.5+ MB
Data Generation took  116.64138746261597  seconds.


In [51]:
df_users.head()

,ID,Surname,Name,Age,Subscription_Date
0,1,Pomme,Thomas,41.73935853733947,1588157171.1168716
1,2,Smith,Alok,54.83159935486658,1588167787.4536881
2,3,Pomme,Thomas,30.86899706338393,1588154407.929673
3,4,Stephenson,Zoe,22.96813536200718,1588159018.037303
4,5,Mueller,Robert,33.13719431632408,1588163405.8847194


In [52]:
df_posts.head()

,PostID,UserID,Content,Timestamp
0,1,87939,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593273255.028971
1,2,39527,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593265232.6049256
2,3,44380,Nam liber tempor cum soluta nobis eleifend opt...,1593277164.7842274
3,4,51170,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593262804.0625734
4,5,73516,"Consetetur sadipscing elitr, sed diam nonumy e...",1593163351.6601837


# Tasks

## Task 1

* __What is the number of unique name combinations?__

* __Who is the oldest user, who is the youngest?__

**Number of Unique Name Combinations**

In [53]:
df_users.groupby(['Surname','Name']).size().reset_index().rename(columns={0:'count'}) #Output Task 1.1

#Grouping on the basis of 'Surnames' and 'Names'
#making a count on each unique combination

,Surname,Name,count
0,Beierlorzer,Agaba,399
1,Beierlorzer,Ali,375
2,Beierlorzer,Alok,400
3,Beierlorzer,Andreas,396
4,Beierlorzer,Anna,363
...,...,...,...
247,Wellington,Sarah,370
248,Wellington,Simon,411
249,Wellington,Thomas,410
250,Wellington,Timothy,388


**Oldest and Youngest User**

In [54]:
print(df_users[['ID','Surname', 'Name','Age']][df_users.Age == df_users.Age.max()]) #Output Task 1.2 Oldest User Acc. to Age

#Extracting data of Oldest user(s) using Max Function

          ID     Surname     Name                Age
95556  95557  Wellington  Andreas  76.53792805861912


In [55]:
print(df_users[['ID','Surname', 'Name','Age']][df_users.Age == df_users.Age.min()]) #Output Task 1.2 Youngest User Acc, to Age

#Extracting data of Youngest user(s) using Min Function

          ID      Surname      Name   Age
5          6        Meier     Agaba  16.0
25        26     Thronton       Zoe  16.0
94        95       Gwahsi  Francine  16.0
147      148  Beierlorzer       Ali  16.0
322      323       Gwahsi       Ali  16.0
...      ...          ...       ...   ...
99787  99788        Smith      Hans  16.0
99793  99794   Wellington   Andreas  16.0
99844  99845       Gwahsi     Josie  16.0
99899  99900  Beierlorzer      Alok  16.0
99927  99928        Smith  Francine  16.0

[2914 rows x 4 columns]


## Task 2

* __Who is the user with most postings?__
* __Who has the least amount of postings?__
* __Which user has "written" most words?__
* __Which one has written the least?__

**User with most and least postings**

In [56]:
df_atp = df_posts.groupby(['UserID']).size().reset_index().rename(columns={0:'TotalPost'})

#establishing a table corresponding to number of post by a user.

In [57]:
df_atp.head()

#Display Table

,UserID,TotalPost
0,1,4
1,10,11
2,100,10
3,1000,14
4,10000,10


In [58]:
df_mostpost = df_atp[df_atp['TotalPost'] == df_atp.TotalPost.max()] 

#Finding user(s) corresponding to most post

In [59]:
df_mostpost.head()

#Display Table

,UserID,TotalPost
98476,98633,27


In [60]:
df_nusers = df_users.rename(columns={'ID':'UserID'})

#renaming 'Id' to 'UserId' which will help in merging and finding the name of User

In [61]:
df_nusers.head() 

#Display table

,UserID,Surname,Name,Age,Subscription_Date
0,1,Pomme,Thomas,41.73935853733947,1588157171.1168716
1,2,Smith,Alok,54.83159935486658,1588167787.4536881
2,3,Pomme,Thomas,30.86899706338393,1588154407.929673
3,4,Stephenson,Zoe,22.96813536200718,1588159018.037303
4,5,Mueller,Robert,33.13719431632408,1588163405.8847194


In [62]:
pd.merge(left=df_mostpost, right=df_nusers, how='left', left_on='UserID', right_on='UserID') #Output Task2.1

#Merging two tables df_user & df_mostpost 
#We get the corresponding values to UserId

,UserID,TotalPost,Surname,Name,Age,Subscription_Date
0,98633,27,Smith,Jordi,21.677202546119606,1588159823.0477908


In [63]:
#we can also use the value obtained in df_mostpost to extract data from df_users as done here

df_users.loc[df_users['ID']=='98633'] #Output Task2.1, Alternative

,ID,Surname,Name,Age,Subscription_Date
98632,98633,Smith,Jordi,21.677202546119606,1588159823.0477908


In [64]:
df_leastpost = df_atp[df_atp['TotalPost'] == df_atp.TotalPost.min()] 

#Finding user(s) corresponding to least post

In [65]:
df_leastpost.head()

#Display Table

,UserID,TotalPost
7487,16737,1
7703,16931,1
8326,17492,1
10316,19283,1
13414,22070,1


In [66]:
pd.merge(left=df_leastpost, right=df_nusers, how='left', left_on='UserID', right_on='UserID') #Output Task2.2

#Merging two tables df_user & df_leastpost 
#We get the corresponding values to UserId

,UserID,TotalPost,Surname,Name,Age,Subscription_Date
0,16737,1,Di Lillo,Lee,31.738598760524827,1588155486.1796212
1,16931,1,Meier,Jordi,16.0,1588150290.2391138
2,17492,1,Pomme,Jordi,33.545689851846944,1588161110.0874844
3,19283,1,Meier,Timothy,21.35443459158268,1588161417.0354567
4,22070,1,Gwahsi,Zoe,28.595237655344043,1588158535.4091601
5,22771,1,Stephenson,Ali,42.51839238070199,1588162144.6393468
6,23193,1,Meier,Ali,39.24516638757797,1588162729.1029968
7,24192,1,Meier,Josie,35.22943747234197,1588153053.6588786
8,28031,1,Beierlorzer,Jordi,40.72799112270901,1588168073.6192417
9,32030,1,Di Lillo,Timothy,31.382926642544646,1588162319.5592277


**User who has "written" most and least words**

In [67]:
df_posts['Totalwords'] = df_posts['Content'].str.count(' ') + 1

#Counting Number of words per post

In [68]:
df_posts.head()

#Display Table with totalwords per post

,PostID,UserID,Content,Timestamp,Totalwords
0,1,87939,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593273255.028971,63
1,2,39527,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593265232.6049256,63
2,3,44380,Nam liber tempor cum soluta nobis eleifend opt...,1593277164.7842274,60
3,4,51170,"Ut wisi enim ad minim veniam, quis nostrud exe...",1593262804.0625734,63
4,5,73516,"Consetetur sadipscing elitr, sed diam nonumy e...",1593163351.6601837,145


In [69]:
df_totalwords = df_posts.groupby(['UserID'])['Totalwords'].sum()

#Adding up totalwords per post corresponding to same User ID

In [70]:
df_totalwords.head()

#Display Table

UserID
1        336
10       770
100      453
1000     836
10000    834
Name: Totalwords, dtype: int64

In [71]:
df_merge = pd.merge(left=df_totalwords, right=df_nusers, how='left', left_on='UserID', right_on='UserID') 

#Merging tables df_nusers and df_totalwords to find the details corresponding to user ID

In [72]:
df_merge.head()

#Display Table

,UserID,Totalwords,Surname,Name,Age,Subscription_Date
0,1,336,Pomme,Thomas,41.73935853733947,1588157171.1168716
1,10,770,Gwahsi,Hans,36.661325747643055,1588148930.10077
2,100,453,Meier,Jean-Luc,45.374033886431334,1588155748.5728843
3,1000,836,Stephenson,Simon,43.00155678042242,1588168475.6795335
4,10000,834,Di Lillo,Josie,41.2415883335132,1588166053.1042876


*In some cases, merging datatype int64 and object results into error. Hence, can use the concat() function.*
e.g df_merge = pd.concat([df_totalwords, df_nusers], axis=1)

In [73]:
df_merge[df_merge['Totalwords'] == df_merge.Totalwords.max()] #Output of Task 2.3

#Extracting user(s) with most words using Max function

,UserID,Totalwords,Surname,Name,Age,Subscription_Date
16847,25160,2319,Bond,Hans,32.18862414449404,1588164650.521033


In [74]:
df_merge[df_merge['Totalwords'] == df_merge.Totalwords.min()] #Output of Task 2.4

#Extracting user(s) with most words using Min function

,UserID,Totalwords,Surname,Name,Age,Subscription_Date
10316,19283,21,Meier,Timothy,21.35443459158268,1588161417.0354567
24481,32030,21,Di Lillo,Timothy,31.382926642544646,1588162319.5592277
47674,52907,21,Bond,Franz,48.29420868033247,1588160039.6411347
48912,54020,21,Stephenson,Hans,43.04336030265533,1588158528.3170936
60753,6468,21,Roth,Timothy,26.205385053779555,1588149506.959423
60837,64755,21,Gwahsi,Robert,26.594444611256158,1588151109.6986656
75193,77677,21,Wellington,Robert,43.962203662942386,1588165326.5878766
90333,91302,21,Kirk,Agaba,36.36773832557818,1588161849.6712987
